In [1]:
import pandas as pd
import re
import json
import matplotlib.pyplot as plt
import gc
from bs4 import BeautifulSoup
from json_stream_parser import load_iter

In [2]:
# convert html to string
def body2text_body(html):
    soup = BeautifulSoup(html, 'lxml')
    result = []
    # kill all script and style elements
    for script in soup(['script', 'style']):
        script.extract()

    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    for line in lines:
        if line:
            result.append(line)

    return result

In [3]:
LIMIT = 500000
i = 0
data = []
date = 'jan-15'
with open('../data/universe-' + date + '.json') as data_file:
    for obj in load_iter(data_file):
        body = obj['_source']['body']
        body_text = body2text_body(body)
        row = [obj['_source']['url'], obj['_source']['title'], body_text, obj['_source']['status']]
        data.append(row)
        if len(data) >= LIMIT:
            print('Output csv ' + str(i))
            df = pd.DataFrame(data, columns=['url', 'title', 'body_text', 'status'])
            df.to_csv('../data/universe-'+date+'-'+str(i)+'.csv', index=False)
            del df
            data = []
            i += 1
            gc.collect()

/home/mwsb/.local/lib/python3.7/site-packages/bs4/__init__.py:282: UserWarning: "http://xu5dw5izkd5acb2z66vkbh2hgkxajtyvj7txg7sttpozpk5mx7nuqcyd.onion/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/mwsb/.local/lib/python3.7/site-packages/bs4/__init__.py:282: UserWarning: "http://u2izrmre7yqzhv5gr3tbbsoyriwoxmc77rfieuz7xdjzjpvk37bhahqd.onion/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/mwsb/.local/lib/python3.7/site-packages/bs4/__init__.py:282: UserWarning: "http://az67bzyxmctkg4mgpob7pdi54ea3tftrnpcvdx6owhdrnvdiqtl4s7id.onion/" looks like a URL. Beautiful Soup is not an HTTP client. Y

Output csv 0
Output csv 1
